In [ ]:
import requests
import csv
import time

# Function to fetch and append data to CSV
def fetch_and_append_data(query, location_uid, csv_filename):
    url = "https://facebook-scraper3.p.rapidapi.com/search/posts"
    querystring = {"query": query, "location_uid": location_uid}

    headers = {
        "x-rapidapi-host": "facebook-scraper3.p.rapidapi.com",
        "x-rapidapi-key": "c7d63f413cmshf2851df4a5e42d1p1ed1bejsn092d86a02e69"
    }

    # Open CSV for appending data (not overwriting)
    with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write headers only once if the file is empty
        file.seek(0, 2)  # Move to the end of the file
        if file.tell() == 0:  # Check if the file is empty
            writer.writerow([
                'post_id', 'type', 'message', 'timestamp', 'comments_count', 'reactions_count', 'author_name', 
                'author_profile', 'author_profile_picture', 'media_type', 'media_url', 'reaction_like', 'reaction_love',
                'reaction_sad', 'reaction_anger', 'reaction_haha', 'reaction_wow'
            ])
            print(f"Writing headers to {csv_filename} as file is empty.")

        last_post_timestamp = None  # Track the last post timestamp

        while True:
            print(f"Fetching new data for query: '{query}' with location UID: '{location_uid}'")
            try:
                response = requests.get(url, headers=headers, params=querystring)
                response.raise_for_status()  # Raise error if response is not 200
                data = response.json()

                # Check if 'results' exists and write new posts
                if data.get('results'):
                    for post in data['results']:
                        post_id = post.get('post_id', '')
                        post_type = post.get('type', '')
                        message = post.get('message', '').replace('\n', ' ').replace('\r', '')
                        timestamp = post.get('timestamp', '')
                        comments_count = post.get('comments_count', 0)
                        reactions_count = post.get('reactions_count', 0)
                        author_name = post.get('author', {}).get('name', '')
                        author_profile = post.get('author', {}).get('profile', '')
                        author_profile_picture = post.get('author', {}).get('profile_picture_url', '')
                        
                        # Handle media info
                        media_type = ""
                        media_url = ""
                        if post.get('media'):
                            media_type = post['media'].get('type', '')
                            media_url = post['media'].get('url', '')
                        
                        # Reactions
                        reactions = post.get('reactions', {})
                        reaction_like = reactions.get('like', 0)
                        reaction_love = reactions.get('love', 0)
                        reaction_sad = reactions.get('sad', 0)
                        reaction_anger = reactions.get('anger', 0)
                        reaction_haha = reactions.get('haha', 0)
                        reaction_wow = reactions.get('wow', 0)

                        # Only write if new post based on timestamp or post_id
                        if last_post_timestamp is None or timestamp > last_post_timestamp:
                            writer.writerow([
                                post_id, post_type, message, timestamp, comments_count, reactions_count, author_name, 
                                author_profile, author_profile_picture, media_type, media_url, reaction_like, reaction_love,
                                reaction_sad, reaction_anger, reaction_haha, reaction_wow
                            ])

                            # Update the last timestamp to track the most recent post
                            last_post_timestamp = timestamp
                            print(f"New post added: {post_id}")

                # Check if there is a cursor to get the next page of data
                cursor = data.get('cursor')
                if cursor:
                    querystring['cursor'] = cursor
                    print("Moving to next page...")
                else:
                    print("No more data. Exiting.")
                    break
            except requests.exceptions.RequestException as e:
                print(f"Request error: {e}. Retrying in 30 seconds.")
            except Exception as e:
                print(f"Unexpected error: {e}. Retrying in 30 seconds.")

            # Sleep for a short interval to prevent constant polling (e.g., 86400 seconds - 24 hours)
            time.sleep(86400)

# Example usage: continuously fetch and append data related to disaster keywords
disaster_keywords = ["flood", "earthquake", "hurricane", "fire", "storm", "tsunami", "disaster", "banjir", "gempa", "taufan"]

for keyword in disaster_keywords:
    fetch_and_append_data(keyword, "malaysia", "facebook_posts.csv")
